In [ ]:
import numpy as np
from sklearn.cluster import KMeans
import torch

In [ ]:
N = 100
tensor = torch.load('../data/resnet50_tensor.pt').numpy().copy()
print('tensor finished')
kmeans_model = KMeans(n_clusters=N, random_state=10).fit(tensor)

# 分類先となったラベルを取得する
labels = kmeans_model.labels_

# ラベル (班) 、成績、三科目の合計得点を表示する
for label, feature in zip(labels, tensor):
    print(label, feature)

In [ ]:
with open(f'cluster_label_{N}.txt', 'w') as f:
    [f.write(f'{l}\n') for l in label]

In [ ]:
center_vectors = np.array([])
for l in range(N):
    center_vector = np.mean(tensor[labels == l], axis = 0)
    center_vectors = np.append(center_vectors, center_vector, axis=0)

center_tensors = torch.from_numpy(center_vectors.astype(np.float32)).clone()
torch.save(center_tensors, './center_tensors.pt')
